# Atención sobre lenguaje matemático usando transformers

In [1]:
import math_dataset 
from math_dataset import MathDatasetManager
import torch
import torch.optim as optim
from torch.utils import data
from math_dataset import (
    question_answer_to_position_batch_collate_fn
)
import model_process


import utils


In [2]:
mdsmgr = MathDatasetManager(
  "C:\\Users\\Jesús\\Documents\\PC2\\TorchDemo\\hs-math-nlp\\mathematics_dataset-v1.0\\mathematics_dataset-v1.0\\"
)

initialized MultiFilesMathDataset with categories ['algebra', 'arithmetic', 'calculus', 'comparison', 'measurement', 'numbers', 'polynomials', 'probability'] and types ['train-easy', 'train-medium', 'train-hard', 'interpolate', 'extrapolate']


In [3]:
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda")
print("device", device)

device cuda


In [4]:
exp_name = "add_sub" # "math_ds_algebra_linear_1d_easy"
unique_id = "2023-07-24" # "2019-05-25_0900"

In [5]:
ds = mdsmgr.build_dataset_from_module(
    'algebra', 'linear_1d', 'train-easy', 1000
)
print("train-easy dataset size", len(ds))

train-easy dataset size 1000


c:\Users\Jesús\Documents\PC2\TorchDemo\venv\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
ds_interpolate = mdsmgr.build_dataset_from_module(
    'algebra', 'linear_1d', 'interpolate', 1000
)
print("interpolate dataset size", len(ds_interpolate))

interpolate dataset size 1000


In [7]:
model = utils.build_transformer()

c:\Users\Jesús\Documents\PC2\TorchDemo\pc5-Atencion-sobre-lenguaje-matematico-usando-transformers\tests\transformer\SubLayers.py:35: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.fc.weight)


In [8]:
optimizer = optim.Adam(model.parameters(), lr=6e-6, betas=(0.9, 0.995), eps=1e-9)

In [9]:
# here we split data in 90/10% for train/validation and use interpolate for test
train_ds, val_ds = math_dataset.random_split_dataset(ds, split_rate=0.9)

# we provide the function question_answer_to_position_batch_collate_fn that collates
# all questions/answers into transformer format enhanced with char positioning
train_loader = data.DataLoader(
    train_ds, batch_size=128, shuffle=True, num_workers=8,
    collate_fn=question_answer_to_position_batch_collate_fn)

val_loader = data.DataLoader(
    val_ds, batch_size=128, shuffle=False, num_workers=8,
    collate_fn=question_answer_to_position_batch_collate_fn)

interpolate_loader = data.DataLoader(
    ds_interpolate, batch_size=128, shuffle=False, num_workers=8,
    collate_fn=question_answer_to_position_batch_collate_fn)

In [10]:
model = model.to(device)


In [11]:
model_process.train(
    name = exp_name +"_" + unique_id,
    model = model,
    training_data= train_loader,
    validation_data = val_loader,
    interpolate_data=interpolate_loader,
    optimizer = optimizer,
    device = device,
    epochs=8,
    tb=None,
    log_interval=100)

~~~ Beginning Training ~~~~
Start epoch: 0, Start batch: 0, Max batch: None
[ Epoch: 0 / 8, Run Batch: 0 / None]
******************************
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([128, 41, 8, 64])
torch.Size([128, 41, 8, 64])
******************************


RuntimeError: batch1 must be a 3D tensor

In [ ]:
c:\Users\Jesús\Documents\PC2\TorchDemo\pc5-Atencion-sobre-lenguaje-matematico-usando-transformers\tests\transformer\Modules.py in forward(self, q, k, v, mask)
     22     def forward(self, q: Tensor, k: Tensor, v: Tensor, mask: Tensor = None) -> Tuple[Tensor]:
     23         # bmm realiza un producto MatMul(Q, K)
---> 24         attn: Tensor = torch.bmm(input=q, mat2=k.transpose(dim0=1, dim1=2))
     25         # Scale
     26         attn: Tensor = attn / self.temperature

